In [1]:
cd /home/jovyan

/home/jovyan


In [2]:
N_CORES = 128 # Где N_CORES - количество ядер CPU

import os

os.environ["MKL_NUM_THREADS"] = str(N_CORES)

os.environ["NUMEXPR_NUM_THREADS"] = str(N_CORES)

os.environ["OMP_NUM_THREADS"] = str(N_CORES)
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline, KandinskyV22PriorEmb2EmbPipeline, KandinskyV22ControlnetImg2ImgPipeline, KandinskyV22Img2ImgPipeline
import torch
from PIL import Image
from PIL import ImageDraw, ImageFont
from copy import deepcopy
from deep_translator import GoogleTranslator

from rembg import remove
import sys
sys.path.append('/home/jovyan/cene655/ControlNet')
from annotator.util import resize_image, HWC3
from annotator.midas import MidasDetector
from diffusers import KandinskyV22PriorEmb2EmbPipeline
from diffusers import KandinskyV22ControlnetImg2ImgPipeline
import torch
import PIL
import torch
from diffusers.utils import load_image
from torchvision import transforms
from transformers import CLIPVisionModelWithProjection
from diffusers.models import UNet2DConditionModel
import numpy as np

/home/user/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/home/user/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:637: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/home/user/conda/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:108: DeprecationWarning: `np.object` i

Extension horovod.torch has not been built: /home/user/conda/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.


In [3]:
def make_hint(img, detector):
    input_image = np.array(img)
    img = HWC3(input_image)#resize_image(, input_image.shape[1])
    print(img.shape)
    H, W, C = img.shape
    detected_map, _ = detector(img)
    detected_map = HWC3(detected_map)
    detected_map = torch.from_numpy(detected_map.copy()).float() / 255.0
    hint = detected_map.permute(2, 0, 1)
    return hint

In [4]:
detector = MidasDetector()

/home/jovyan/.local/lib/python3.7/site-packages/timm/models/_factory.py:118: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  **kwargs,


In [5]:
decoder = KandinskyV22Img2ImgPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-decoder', torch_dtype=torch.float16).to('cuda')

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
controlnet = KandinskyV22ControlnetImg2ImgPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-controlnet-depth', torch_dtype=torch.float16).to('cuda')

movq/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
prior = KandinskyV22PriorPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior', torch_dtype=torch.float16).to('cuda')

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
def prepare_caption(caption): return GoogleTranslator(source='ru', target='en').translate(caption)

In [9]:
def delete_bg(img, img2):
    min_size = min(img2.size[0], img2.size[1])

    img = img.resize((min_size, min_size))
    img = np.array(img)
    img2 = np.array(img2)
    output = remove(img)
    m = 0
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            m = max(output[i][j][3], m)
            if output[i][j][3] >= 80:
                img2[i][j] = output[i][j][:3]
    return Image.fromarray(img2)

In [26]:
def generate_avatar(decoder, controlnet, prior, prompt, text=None, style_image=None, face_image=None, animation=None):
        
    zero_img = PIL.Image.new(mode="RGB", size=(200, 200))
    img_emb = prior(prompt=prompt, num_inference_steps=50, num_images_per_prompt=1, guidance_scale=1.0).image_embeds
    if style_image is not None:
        img_emb2 = prior.interpolate(images_and_prompts=[style_image], weights=[1.0], num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0).image_embeds
        img_emb = img_emb * 0.55 + img_emb2 * 0.45
    negative_emb = prior(prompt='blow quility', num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0)
    images = decoder(image=zero_img, strength=1.0, image_embeds=img_emb, negative_image_embeds=negative_emb.negative_image_embeds, num_inference_steps=100, height=832, width=832, guidance_scale=4.0).images
    if face_image is not None:
        images[0] = delete_bg(face_image, images[0])
    if text is not None:
        img = deepcopy(images[0])
        W, H = img.size
        myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 60)

        draw = ImageDraw.Draw(img)
        _, _, w, h = draw.textbbox((0, 0), text, font=myFont)
        draw.text(((W-w)/2, (H-h)/2), text, font=myFont, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0))
        hint = make_hint(img, detector).unsqueeze(0).half().to('cuda').repeat(1, 1, 1, 1)
        images2 = controlnet(image=img, strength=0.05, image_embeds=img_emb, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=100, height=img.size[1], width=img.size[0])
        return images2[0][0].resize((800, 800))
    else:
        return images[0].resize((800, 800))

    

In [13]:
def generate_preview(decoder, controlnet, prior, prompt, text=None, style_image=None, face_image=None, animation=None):
    prompt = '4k photo of ' + prompt
    zero_img = PIL.Image.new(mode="RGB", size=(200, 200))
    img_emb = prior(prompt=prompt, num_inference_steps=50, num_images_per_prompt=1, guidance_scale=1.5).image_embeds
    if style_image is not None:
        img_emb2 = prior.interpolate(images_and_prompts=[style_image], weights=[1.0], num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0).image_embeds
        img_emb = img_emb * 0.75 + img_emb2 * 0.25
    negative_emb = prior(prompt='blow quility', num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0)
    images = decoder(image=zero_img, strength=1.0, image_embeds=img_emb, negative_image_embeds=negative_emb.negative_image_embeds, num_inference_steps=250, height=704, width=1280, guidance_scale=4.0).images
    if face_image is not None:
        images[0] = delete_bg(face_image, images[0])
    if text is not None:
        img = deepcopy(images[0])
        W, H = img.size
        if len(text) < 52:
            myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 50)
        else:
            myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 35)
        draw = ImageDraw.Draw(img)
        _, _, w, h = draw.textbbox((0, 0), text, font=myFont)
        draw.text(((W-w)/2, 25), text, font=myFont, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0))
        hint = make_hint(img, detector).unsqueeze(0).half().to('cuda').repeat(1, 1, 1, 1)
        #images2 = controlnet(image=img, strength=0.01, image_embeds=img_emb, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=150, height=img.size[1], width=img.size[0])
        return img#images2[0][0]
    else:
        return images[0]

    

In [14]:
def generate_banner(decoder, controlnet, prior, prompt, text=None, style_image=None, face_image=None, animation=None):
    prompt = '4k photo of ' + prompt
    zero_img = PIL.Image.new(mode="RGB", size=(200, 200))
    img_emb = prior(prompt=prompt, num_inference_steps=50, num_images_per_prompt=1, guidance_scale=1.5).image_embeds
    if style_image is not None:
        img_emb2 = prior.interpolate(images_and_prompts=[style_image], weights=[1.0], num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0).image_embeds
        img_emb = img_emb * 0.75 + img_emb2 * 0.25
    negative_emb = prior(prompt='blow quility', num_inference_steps=25, num_images_per_prompt=1, guidance_scale=4.0)
    images = decoder(image=zero_img, strength=1.0, image_embeds=img_emb, negative_image_embeds=negative_emb.negative_image_embeds, num_inference_steps=100, height=896, width=2240, guidance_scale=4.0).images
    if face_image is not None:
        images[0] = delete_bg(face_image, images[0])
    if text is not None:
        img = deepcopy(images[0])
        W, H = img.size
        if len(text) < 52:
            myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 100)
        else:
            myFont = ImageFont.truetype('cene655/drawbench/DejaVuSansCondensed.ttf', 35)
        draw = ImageDraw.Draw(img)
        _, _, w, h = draw.textbbox((0, 0), text, font=myFont)
        draw.text(((W-w)/2, (H-h)/2), text, font=myFont, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0))
        hint = make_hint(img, detector).unsqueeze(0).half().to('cuda').repeat(1, 1, 1, 1)
        images2 = controlnet(image=img, strength=0.05, image_embeds=img_emb, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=150, height=img.size[1], width=img.size[0])
        return images2[0][0].resize((2204, 864))
    else:
        return images[0].resize((2204, 864))
    

In [17]:
import pandas as pd
df = pd.read_csv('cene655/videos_c/rutube_hackathon_sochi/data.csv')

In [18]:
df

,title,description,channel_title,video_name
0,Антон Протеинов I #1 IКак накачать все мышцы з...,В этой серии Легенда ворк-аута Антон Протеинов...,Антон ПРОТЕИНОВ,0.mp4
1,Антон Протеинов I #2 I Как выжать максимум из ...,В этой серии Легенда ворк аута Антон Протеинов...,Антон ПРОТЕИНОВ,1.mp4
2,Антон Протеинов I #3 I Как правильно заходить ...,В этом выпуске вы узнаете от Легенды мирового ...,Антон ПРОТЕИНОВ,2.mp4
3,Антон Протеинов I #4 I Самые эффективные упраж...,В этом выпуске Антон Протеинов познакомит вас ...,Антон ПРОТЕИНОВ,3.mp4
4,"Антон Протеинов I #5 I Как встать в 3 утра, чт...","В этом выпуске, легенда ворк аута Антон Протеи...",Антон ПРОТЕИНОВ,4.mp4
...,...,...,...,...
95,МультНайтШоу. Выпуск 5. В гостях Ивлеева. Прос...,МультНайтШоу – еженедельное юмористическое нар...,МультНайтШоу,95.mp4
96,МультНайтКлип. Удалите маркетплейсы,Специальное включение с «черной пятницы» от му...,МультНайтШоу,96.mp4
97,МультНайтШоу. Выпуск 6. В гостях Джокер. Наши ...,МультНайтШоу – еженедельное юмористическое нар...,МультНайтШоу,97.mp4
98,МультНайтШоу. Выпуск 7. В гостях Нагиев. Ю вон...,МультНайтШоу – еженедельное юмористическое нар...,МультНайтШоу,98.mp4


In [16]:
import imageio.v3 as iio
for i in range( len(df)):
    frame = Image.fromarray(iio.imread(
                'cene655/videos_c/rutube_hackathon_sochi/videos/' + df['video_name'].iloc[i],
                index=1000,
                plugin="pyav",
            ))
    out_img = generate_preview(decoder, controlnet, prior, prepare_caption(df['description'].iloc[i]), text=df['title'].iloc[i].replace('I', '\n').replace('|', '\n'), style_image=frame, face_image=None, animation=None)
    out_img.save(f'cene655/videos_c/avatars/{i}.jpg')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 import imageio.v3 as iio                                                                    │
│ ❱  2 for i in range( len(df)):                                                                   │
│    3 │   frame = Image.fromarray(iio.imread(                                                     │
│    4 │   │   │   │   'cene655/videos_c/rutube_hackathon_sochi/videos/' + df['video_name'].ilo    │
│    5 │   │   │   │   index=1000,                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df' is not defined

In [27]:
import imageio.v3 as iio
for i in range(0, len(df)):
    frame = Image.fromarray(iio.imread(
                'cene655/videos_c/rutube_hackathon_sochi/videos/' + df['video_name'].iloc[i],
                index=1000,
                plugin="pyav",
            ))
    out_img = generate_avatar(decoder, controlnet, prior, prepare_caption(df['description'].iloc[i]), text=df['channel_title'].iloc[i].replace('I', '\n').replace('|', '\n'), style_image=frame, face_image=None, animation=None)
    out_img.save(f'cene655/videos_c/avatars/{i}.jpg')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

(832, 832, 3)


...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

(832, 832, 3)


  0%|          | 0/5 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 │   │   │   │   index=1000,                                                                 │
│    6 │   │   │   │   plugin="pyav",                                                              │
│    7 │   │   │   ))                                                                              │
│ ❱  8 │   out_img = generate_avatar(decoder, controlnet, prior, prepare_caption(df['descriptio    │
│    9 │   out_img.save(f'cene655/videos_c/avatars/{i}.jpg')                                       │
│   10                                                                                             │
│                                                                                                  │
│ in prepare_caption:1                                                                             │
│                                                                                                  │
│ ❱ 1 def prepare_caption(caption): return GoogleTranslator(source='ru', target='en').translat     │
│   2                                                                                              │
│                                                                                                  │
│ /home/jovyan/.imgenv-cene655-test2-video-0/lib/python3.7/site-packages/deep_translator/google.py │
│ :57 in translate                                                                                 │
│                                                                                                  │
│    54 │   │   @param text: desired text to translate                                             │
│    55 │   │   @return: str: translated text                                                      │
│    56 │   │   """                                                                                │
│ ❱  57 │   │   if is_input_valid(text, max_chars=5000):                                           │
│    58 │   │   │   text = text.strip()                                                            │
│    59 │   │   │   if self._same_source_target() or is_empty(text):                               │
│    60 │   │   │   │   return text                                                                │
│                                                                                                  │
│ /home/jovyan/.imgenv-cene655-test2-video-0/lib/python3.7/site-packages/deep_translator/validate. │
│ py:39 in is_input_valid                                                                          │
│                                                                                                  │
│   36 │   """                                                                                     │
│   37 │                                                                                           │
│   38 │   if not isinstance(text, str):                                                           │
│ ❱ 39 │   │   raise NotValidPayload(text)                                                         │
│   40 │   if max_chars and (not min_chars <= len(text) < max_chars):                              │
│   41 │   │   raise NotValidLength(text, min_chars, max_chars)                                    │
│   42                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotValidPayload: nan --> text must be a valid text with maximum 5000 character,otherwise it cannot be translated